In [1]:
using Turing
using Random
using Plots
using Distributions
using StatsPlots
using Interact
using MCMCChains

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-3953719693773906224\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-8342690290512000471\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-3953719693773906224\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

In [2]:
prob = 0.5
Ns = 0:100

0:100

In [3]:
Random.seed!(12)
data = rand(Bernoulli(prob), last(Ns));

In [4]:
prior = Beta(1, 1)

Beta{Float64}(α=1.0, β=1.0)

In [5]:
animation = @animate for (i, N) in enumerate(Ns)
    heads = sum(data[1:i-1])
    tails = N - heads
    
    updated_belief = Beta(prior.α + heads, prior.β + tails)
    
    plot(updated_belief,
        size = (500, 250),
        title = "Updated belief after $N observations",
        xlabel = "Probability of heads",
        ylabel = "",
        legend = nothing,
        xlim = (0, 1),
        fill = 0, α = 0.3, w = 3)
    vline!([prob])
end;

gif(animation, "anim.gif", fps = 20)

┌ Info: Saved animation to 
│   fn = /mnt/Data/Repos/Julia_stuff/anim.gif
└ @ Plots /home/alex/.julia/packages/Plots/Iuc9S/src/animation.jl:95


Plots.AnimatedGif("/mnt/Data/Repos/Julia_stuff/anim.gif")

In [6]:
@model coinflip(y) = begin
    p ~ Beta(1, 1)
    
    N = length(y)
    for n in 1:N
        y[n] ~ Bernoulli(p)
    end
end;

In [7]:
iterations = 1000
ϵ = 0.05
τ = 10

chain = sample(coinflip(data), HMC(iterations, ϵ, τ))

MethodError: MethodError: no method matching HMC{Turing.Core.ForwardDiffAD{40},space,metricT} where metricT<:AdvancedHMC.Adaptation.AbstractMetric where space(::Int64, ::Float64, ::Int64)

In [ ]:
p_summ = chain[:p]

In [ ]:
plot(p_summ, seriestype = :histogram, bins = 50)

In [ ]:
N = length(data)
heads = sum(data)
updated_belief = Beta(prior.α + heads, prior.β + N - heads)

p = plot(p_summ, seriestype = :density, xlim = (0, 1), legend = :best, w = 2, c = :blue)

In [ ]:
plot!(p, 
    range(0, stop = 1, length = 100), 
    pdf.(Ref(updated_belief), range(0, stop = 1, length = 100)),
    xlabel = "Probability of heads", ylabel = "", title = "",
    xlim = (0, 1), label = "Closed-form",
    fill = 0, α = 0.3, w = 3, c = :lightgreen)

In [ ]:
vline!(p, [prob], label = "True Probability", c = :red)